In [ ]:
import typing

import rich
import dotenv

import huggingface_hub

from twon_lss.schemas import Post, User, Feed
from twon_lss.utility import LLM, Message

from twon_lss.agent.twon_agent import Agent, AgentInstructions, AgentActions

In [23]:
CFG = dotenv.dotenv_values("../.env")

MODEL=LLM(
    client=huggingface_hub.InferenceClient(api_key=CFG["HF_TOKEN"]),
    model="meta-llama/Meta-Llama-3-8B-Instruct",
)

In [24]:
INDIVIDUALS = dict(
    progressive=dict(
        persona="Act as a social media user who embodies progressive American activist viewpoints. Your character should have a consistent posting and commenting history.",
        post="Proud to join the climate rally downtown today! Over 500 people showed up demanding our city divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs - we just need leaders brave enough to fight for it!"
    ),
    moderate=dict(
        persona="Act as a social media user who embodies moderate American centrist viewpoints. Your character should have a consistent posting and commenting history.",
        post="Both parties are missing the point on healthcare reform. Instead of screaming about 'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding."
    ),
    libertarian=dict(
        persona="Act as a social media user who embodies libertarian American political viewpoints. Your character should have a consistent posting and commenting history.",
        post="Another day, another government program that costs billions and delivers nothing. TSA security theater, NSA surveillance, endless foreign wars - when did we decide the federal government should control every aspect of our lives? Less government, more freedom. That's not radical, that's constitutional."
    ),
    conservative=dict(
        persona="Act as a social media user who embodies mainstream American conservative viewpoints. Your character should have a consistent posting and commenting history.",
        post="Just filled up my truck - $3.89/gallon! Remember when gas was under $2 during Trump's presidency? This administration's war on American energy is hitting working families hard. We need to get back to energy independence and stop begging other countries for oil. "
    )
)

GLOBAL_INSTRUCTIONS: typing.Dict[str, str] = dict(
    select_actions="Choose one or multiple of the following actions you would perform on the following post and return them separated by colons. Respond with the action names only.",
    comment="Write a short comment considering the following original post and past comments.",
    post="Write a independent post considering the following content.",
)

In [39]:
USERS = [User(id=f"U{n}") for n in range(len(INDIVIDUALS))]
rich.print(USERS)

[User(id='U0'), User(id='U1'), User(id='U2'), User(id='U3')]

In [40]:
feed = Feed([
    Post(id=f"P{n}", user=user, content=individual["post"])
    for n, (user, individual) in enumerate(zip(USERS, INDIVIDUALS.values()))
])
rich.print(feed)

Feed(
    root=[
        Post(
            id='P0',
            user=User(id='U0'),
            content='Proud to join the climate rally downtown today! Over 500 people showed up demanding our city 
divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs -
we just need leaders brave enough to fight for it!',
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 7, 23, 16, 4, 56, 73387)
        ),
        Post(
            id='P1',
            user=User(id='U1'),
            content="Both parties are missing the point on healthcare reform. Instead of screaming about 
'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, 
prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding.",
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 7, 23, 16, 4, 56, 73400)
        ),
        Post(
            id='P2',
            user=User(id='U2'),
            content="Another day, another government program that costs billions and delivers nothing. TSA security
theater, NSA surveillance, endless foreign wars - when did we decide the federal government should control every 
aspect of our lives? Less government, more freedom. That's not radical, that's constitutional.",
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 7, 23, 16, 4, 56, 73403)
        ),
        Post(
            id='P3',
            user=User(id='U3'),
            content="Just filled up my truck - $3.89/gallon! Remember when gas was under $2 during Trump's 
presidency? This administration's war on American energy is hitting working families hard. We need to get back to 
energy independence and stop begging other countries for oil. ",
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 7, 23, 16, 4, 56, 73406)
        )
    ]
)

In [44]:
individuals = {
    user: Agent(llm=MODEL, instructions=AgentInstructions(persona=individual["persona"], **GLOBAL_INSTRUCTIONS), memory=[Message(role="system", content=post.content)])
    for user, individual, post in zip(USERS, INDIVIDUALS.values(), feed.root)
}
rich.print(individuals)

{
    User(id='U0'): Agent(
        llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct'),
        instructions=AgentInstructions(
            persona='Act as a social media user who embodies progressive American activist viewpoints. Your 
character should have a consistent posting and commenting history.',
            select_actions='Choose one or multiple of the following actions you would perform on the following post
and return them separated by colons. Respond with the action names only.',
            comment='Write a short comment considering the following original post and past comments.',
            post='Write a independent post considering the following content.'
        ),
        action_likelihoods={
            <AgentActions.read: 'read'>: 1.0,
            <AgentActions.like: 'like'>: 1.0,
            <AgentActions.comment: 'comment'>: 1.0,
            <AgentActions.post: 'post'>: 1.0
        },
        memory=[
            Message(
                role='system',
                content='Proud to join the climate rally downtown today! Over 500 people showed up demanding our 
city divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green 
jobs - we just need leaders brave enough to fight for it!'
            )
        ],
        memory_length=4
    ),
    User(id='U1'): Agent(
        llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct'),
        instructions=AgentInstructions(
            persona='Act as a social media user who embodies moderate American centrist viewpoints. Your character 
should have a consistent posting and commenting history.',
            select_actions='Choose one or multiple of the following actions you would perform on the following post
and return them separated by colons. Respond with the action names only.',
            comment='Write a short comment considering the following original post and past comments.',
            post='Write a independent post considering the following content.'
        ),
        action_likelihoods={
            <AgentActions.read: 'read'>: 1.0,
            <AgentActions.like: 'like'>: 1.0,
            <AgentActions.comment: 'comment'>: 1.0,
            <AgentActions.post: 'post'>: 1.0
        },
        memory=[
            Message(
                role='system',
                content="Both parties are missing the point on healthcare reform. Instead of screaming about 
'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, 
prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding."
            )
        ],
        memory_length=4
    ),
    User(id='U2'): Agent(
        llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct'),
        instructions=AgentInstructions(
            persona='Act as a social media user who embodies libertarian American political viewpoints. Your 
character should have a consistent posting and commenting history.',
            select_actions='Choose one or multiple of the following actions you would perform on the following post
and return them separated by colons. Respond with the action names only.',
            comment='Write a short comment considering the following original post and past comments.',
            post='Write a independent post considering the following content.'
        ),
        action_likelihoods={
            <AgentActions.read: 'read'>: 1.0,
            <AgentActions.like: 'like'>: 1.0,
            <AgentActions.comment: 'comment'>: 1.0,
            <AgentActions.post: 'post'>: 1.0
        },
        memory=[
            Message(
                role='system',
                content="Another day, another government program that costs billions and delivers nothing. TSA 
security theater, NSA surveillance, e

In [ ]:
for user, agent in individuals.items():
    for item in feed.root:
        selected_actions = agent.select_actions(item)
        rich.print(user.id, item.id, selected_actions)

        if AgentActions.comment in selected_actions:
            rich.print(agent.comment(item))

U0 P0
[<AgentActions.read: 'read'>, <AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

U0 P1
[<AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

U0 P2
[<AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

U0 P3
[<AgentActions.comment: 'comment'>]

U1 P0
[<AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

...

U2 P3
[<AgentActions.read: 'read'>, <AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

U3 P0
[<AgentActions.comment: 'comment'>]

U3 P1
[<AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

U3 P2
[<AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

U3 P3
[
    <AgentActions.read: 'read'>,
    <AgentActions.like: 'like'>,
    <AgentActions.like: 'like'>,
    <AgentActions.comment: 'comment'>
]